In [1]:
import argparse
import os
import requests
import json
import datetime
import pandas as pd

 

parser = argparse.ArgumentParser()
parser.add_argument('--scrape_iter',
    required=False,
    help='Scrape Iteration')
parser.add_argument('--concat_df',
    required=True,
    choices=['Y', 'N'],
    help='Use this script to concat all of the extracted datasets or to extract StockTwits data part by part')
parser.add_argument('--path',
    required=True,
    help="Path to SENN folder")

 


def stocktwits_scrap(n,ticker,base=199407261):
    '''
    Function to scrap stocktwits tweet

 

    Base: Base ID to scrape from
    '''

 

    id=[]
    created_at=[]
    body=[]
    senti=[]
    for i in range(n):
        try:
            url = "https://api.stocktwits.com/api/2/streams/symbol/"+ticker+".json?max="+str(base+5000)+"&since="+str(base)+"&limit=30"

 

            base=base+5001

 

            response = requests.request("GET", url, headers={}, data ={})
            response=response.json()
            messages=response['messages']
            iter=range(len(messages))
            for idx in iter:
                id.append(messages[idx]['id'])
                created_at.append(messages[idx]['created_at'])
                body.append(messages[idx]['body'])
                try:
                    senti.append(messages[idx]['entities']['sentiment']['basic']) 
                except:
                    senti.append(' ')
                                    
                if (len(id) % 100) == 0:
                    print('Done Scrape ',len(id),' messages.')
        except:
            print('Warning Messages: \n ID Out of Range or Too Many Requests within 1 Hour')
            break
    try:
        df_stocktwits=pd.DataFrame(id,columns=['id'])
        df_stocktwits['created_at']=created_at
        df_stocktwits['created_at']=pd.to_datetime(df_stocktwits['created_at'])
        df_stocktwits['body']=body
        df_stocktwits['senti']=senti
        
        df_stocktwits=df_stocktwits.sort_values(by='created_at')
        df_stocktwits=df_stocktwits.reset_index(drop=True)
        print('Done Scrape ',len(id),' messages.')
        print('Last ID: ',df_stocktwits.tail(1)['id'].values[0])
        return(df_stocktwits,df_stocktwits.tail(1)['id'].values[0])
    except:
        print('Error')
        return None

In [2]:
df=stocktwits_scrap(10,'TSLA',base=220407261)

Done Scrape  100  messages.
Done Scrape  200  messages.
Done Scrape  287  messages.
Last ID:  220456914


In [ ]:
df[0].to_csv('StocktTwits_TSLA_1.csv')